In [1]:
# Importing important modules and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from statistics import stdev
from scipy.stats import t

In [2]:
data = pd.read_csv('C:/Users/LENOVO/Desktop/Task1.csv')

EDA has already been performed on this dataset. 

In [3]:
data.shape

(88827, 12)

In [330]:
data.head(7)

,DATE,STORE_NBR,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,BRAND,PACK_SIZE,TOT_SALES
0,2018-10-17,1,1000,YOUNG SINGLES/COUPLES,Premium,1,5,Natural Chip Compny SeaSalt175g,2,Natural Chip Company,175g,6.0
1,2019-05-14,1,1307,MIDAGE SINGLES/COUPLES,Budget,348,66,CCs Nacho Cheese 175g,3,CCs,175g,6.3
2,2019-05-20,1,1343,MIDAGE SINGLES/COUPLES,Budget,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,Smiths,170g,2.9
3,2018-08-17,2,2373,MIDAGE SINGLES/COUPLES,Budget,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,Smiths,175g,15.0
4,2018-08-18,2,2426,MIDAGE SINGLES/COUPLES,Budget,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,Kettle,150g,13.8
5,2019-05-16,4,4149,MIDAGE SINGLES/COUPLES,Budget,3333,16,Smiths Crinkle Chips Salt & Vinegar 330g,1,Smiths,330g,5.7
6,2018-08-20,5,5026,MIDAGE SINGLES/COUPLES,Budget,4525,42,Doritos Corn Chip Mexican Jalapeno 150g,1,Doritos,150g,3.9


Let's find out the total number of stores

In [5]:
len(data['STORE_NBR'].unique())

267

In [7]:
df = data.copy()

Extracting month and year from all the data points in the dataframe

In [8]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['MONTH_YEAR'] = pd.to_datetime(df['DATE']).dt.strftime('%Y-%m')

In [9]:
df['MONTH_YEAR'].head()

0    2018-10
1    2019-05
2    2019-05
3    2018-08
4    2018-08
Name: MONTH_YEAR, dtype: object

In [10]:
df['MONTH_YEAR'].dtypes

dtype('O')

In [13]:
df1 = df.groupby('STORE_NBR')['MONTH_YEAR'].nunique()

df1 = df1.to_frame()

In [14]:
df2 = df1[df1['MONTH_YEAR'] != 12]

In [15]:
df2.index.values

array([ 11,  14,  31,  42,  44,  52,  92,  99, 117, 127, 132, 139, 140,
       146, 158, 159, 161, 177, 192, 198, 204, 206, 218, 224, 244, 258,
       263, 267], dtype=int64)

In [17]:
for index in df2.index.values:
    df = df[df['STORE_NBR'] != index]

In [18]:
len(df['STORE_NBR'].unique())

239

Now the dataset only consists of stores which made sales in all the months from June 2018 to July 2019

Now we will create the metrics that will be used for firther analysis of the trial and control stores. 

The metrics include: 
    1. Total Sales for each store in each month 
    2. Number of customers in each store 
    3. Transactions per Customer in each store for every month 
    4. Chips bought per customer 
    5. Average unit Price 

In [111]:
metric = df.groupby(['STORE_NBR', 'MONTH_YEAR'])['TOT_SALES', 'PROD_QTY'].sum()

In [112]:
j = df.groupby(['STORE_NBR', 'MONTH_YEAR'])['LYLTY_CARD_NBR'].nunique()

j = j.to_frame()

In [113]:
metric = pd.merge(left=metric, right=j, on=['MONTH_YEAR', 'STORE_NBR'])

In [114]:
metric['Transaction_per_customer'] = metric['TOT_SALES']/metric['LYLTY_CARD_NBR']
metric['Chips_per_customer'] = metric['PROD_QTY']/metric['LYLTY_CARD_NBR']

metric['Averagr Price'] = metric['TOT_SALES']/metric['PROD_QTY']

In [115]:
metric.head(10)

,,TOT_SALES,PROD_QTY,LYLTY_CARD_NBR,Transaction_per_customer,Chips_per_customer,Averagr Price
MONTH_YEAR,STORE_NBR,,,,,,
2018-07,1,77.6,24,22,3.527273,1.090909,3.233333
2018-08,1,46.0,15,14,3.285714,1.071429,3.066667
2018-09,1,95.7,27,22,4.350000,1.227273,3.544444
2018-10,1,81.2,25,17,4.776471,1.470588,3.248000
2018-11,1,84.7,27,19,4.457895,1.421053,3.137037
2018-12,1,35.8,12,10,3.580000,1.200000,2.983333
2019-01,1,36.3,10,8,4.537500,1.250000,3.630000
2019-02,1,65.4,21,18,3.633333,1.166667,3.114286
2019-03,1,90.5,26,21,4.309524,1.238095,3.480769


In [116]:
metric.rename(columns={'LYLTY_CARD_NBR':'Customer_per_store', 'TOT_SALES':'Total Sales', 'PROD_QTY':'Quantity'}, 
             inplace=True)

In [117]:
metric.head(10)

,,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
MONTH_YEAR,STORE_NBR,,,,,,
2018-07,1,77.6,24,22,3.527273,1.090909,3.233333
2018-08,1,46.0,15,14,3.285714,1.071429,3.066667
2018-09,1,95.7,27,22,4.350000,1.227273,3.544444
2018-10,1,81.2,25,17,4.776471,1.470588,3.248000
2018-11,1,84.7,27,19,4.457895,1.421053,3.137037
2018-12,1,35.8,12,10,3.580000,1.200000,2.983333
2019-01,1,36.3,10,8,4.537500,1.250000,3.630000
2019-02,1,65.4,21,18,3.633333,1.166667,3.114286
2019-03,1,90.5,26,21,4.309524,1.238095,3.480769


In [118]:
metric_data = metric.copy()

In [119]:
trial = metric.copy()

In [120]:
metric_data = metric_data.loc[(['2018-07', '2018-08', '2018-09', '2018-10', '2018-11', 
                                '2018-12', '2019-01'], slice(None)), :]

In [29]:
metric.shape

(2868, 6)

In [30]:
metric_data.shape

(1673, 6)

Metric includes observations for the complete span of one year

Metric_data includes observations only till pre-trial period

Correlation of Potential Control stores to the trial stores

In [31]:
metric_data1 = metric_data.copy()

In [32]:
metric_data1.drop([77,86,88], level=1, axis=0, inplace=True)

In [33]:
metric_data1.shape

(1652, 6)

The trial stores have been dropped from the matrix of potential control stores

In [34]:
trial = metric.loc[(['2019-02', '2019-03', '2019-04'], slice(None)), :]
trial = trial.loc[(slice(None), [77, 86, 88]), :]

In [35]:
trial

,,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
MONTH_YEAR,STORE_NBR,,,,,,
2019-02,77,75.8,25,16,4.737500,1.562500,3.032000
2019-03,77,72.2,24,15,4.813333,1.600000,3.008333
2019-04,77,109.8,29,18,6.100000,1.611111,3.786207
2019-02,86,350.4,102,46,7.617391,2.217391,3.435294
2019-03,86,332.6,88,42,7.919048,2.095238,3.779545
2019-04,86,323.8,94,45,7.195556,2.088889,3.444681
2019-02,88,393.2,94,46,8.547826,2.043478,4.182979
2019-03,88,520.4,122,61,8.531148,2.000000,4.265574
2019-04,88,468.6,108,50,9.372000,2.160000,4.338889


The trial dataset contains the data for trial stores in the period of Feb 2019 - April 2019

In [36]:
trial_1 = metric_data.loc[(['2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01'], [77, 86, 88]), :]

#### Table Summary

'Metric' contains all the data <br>
'Metric_data' contains the data before the trial period <br>
'trial' contains data for trial stores in the trial period <br>
'trial_1' contains data for trial stores in the pre-trial duration <br>
'metric_data1' contains data for all the stores except the trial stores before the pre-trial method

In [121]:
tables = [trial_1, metric_data, metric, trial, metric_data1]

for table in tables:
    table = table.reset_index(inplace=True)

In [122]:
cols = ['STORE_NBR' ,'MONTH_YEAR', 'Total Sales', 'Quantity',
       'Customer_per_store', 'Transaction_per_customer', 'Chips_per_customer',
       'Averagr Price']

In [123]:
metric = metric[cols]

In [124]:
metric_data1 = metric_data1[cols]
metric_data = metric_data[cols]
trial = trial[cols]
trial_1 = trial_1[cols]

Creating a function which calculates the correlation between trial and control stores based on a single metric

In [54]:
def calcorr(inputTable, metrics, trial_store):
    
    output = pd.DataFrame({'Store1':[], 'Store2':[], 'Correlation':[]})
    
    a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metrics]
    a.reset_index(drop=True, inplace=True)
    
    Store_Numbers = inputTable['STORE_NBR'].unique()
    
    for i in Store_Numbers:
        b = inputTable.loc[inputTable['STORE_NBR'] == i, metrics]
        b.reset_index(drop=True, inplace=True)
        
        output = output.append({'Store1':trial_store, 'Store2':i, 'Correlation':b.corr(a)}, ignore_index=True)
        
    return output

Creating a function which calculates the standardized magnitude difference

In [55]:
def caldist(inputTable, metrics, trial_store):
    
    output = pd.DataFrame({'Store1':[], 'Store2':[], 'Magnitude':[]})
    
    a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metrics]
    a.reset_index(drop=True, inplace=True)
    
    Store_Numbers = inputTable['STORE_NBR'].unique()
    
    for i in Store_Numbers:
        b = inputTable.loc[inputTable['STORE_NBR'] == i, metrics]
        b.reset_index(drop=True, inplace=True)
        
        c = abs(a-b)
        d = np.mean(1-(c-min(c))/(max(c)-min(c)))
        
        output = output.append({'Store1':trial_store, 'Store2':i, 'Magnitude':d}, ignore_index=True)
        
    return output

Selecting Control Store for trial store 77

In [76]:
trial_store = 77

corr_sales = calcorr(metric_data, 'Total Sales', trial_store)
corr_customer = calcorr(metric_data, 'Customer_per_store', trial_store)
corr_avg_price = calcorr(metric_data, 'Averagr Price', trial_store)

mag_sales = caldist(metric_data, 'Total Sales', trial_store)
mag_customer = caldist(metric_data, 'Customer_per_store', trial_store)
mag_avg_price = caldist(metric_data, 'Averagr Price', trial_store)

score_sales = pd.concat([corr_sales, mag_sales['Magnitude']], axis=1)

# Adding an additional column in 'score_sales' which calculates the weighted average

corr_weight = 0.5

score_sales['Sales Score'] = corr_weight*score_sales['Correlation'] + (1-corr_weight)*score_sales['Magnitude']
score_sales.head(10)

,Store1,Store2,Correlation,Magnitude,Sales Score
0,77.0,1.0,-0.341275,0.403486,0.031106
1,77.0,2.0,-0.110906,0.410989,0.150041
2,77.0,3.0,-0.134088,0.485765,0.175839
3,77.0,4.0,-0.364167,0.403795,0.019814
4,77.0,5.0,-0.530441,0.581823,0.025691
5,77.0,6.0,-0.083126,0.494748,0.205811
6,77.0,7.0,-0.187441,0.459983,0.136271
7,77.0,8.0,0.479138,0.493050,0.486094
8,77.0,9.0,-0.343593,0.567736,0.112071
9,77.0,10.0,-0.322208,0.530270,0.104031


We have added the weight column to determine whether 'correlation' or 'magnitude'  has a higher weight

In [77]:
score_customer = pd.concat([corr_customer, mag_customer['Magnitude']], axis=1)

score_customer['Customer Score'] = corr_weight*score_customer['Correlation'] + (1-corr_weight)*score_customer['Magnitude']
score_customer.head(10)

,Store1,Store2,Correlation,Magnitude,Customer Score
0,77.0,1.0,0.205764,0.587302,0.396533
1,77.0,2.0,-0.224065,0.482143,0.129039
2,77.0,3.0,-0.081832,0.428571,0.173370
3,77.0,4.0,-0.176079,0.450000,0.136960
4,77.0,5.0,-0.278693,0.511278,0.116292
5,77.0,6.0,0.304114,0.547619,0.425867
6,77.0,7.0,-0.072336,0.321429,0.124546
7,77.0,8.0,0.143385,0.547619,0.345502
8,77.0,9.0,-0.378962,0.632653,0.126846
9,77.0,10.0,-0.620336,0.470899,-0.074718


In [78]:
score_avg_price = pd.concat([corr_avg_price, mag_avg_price['Magnitude']], axis=1)

score_avg_price['Avg Price Score'] = corr_weight*score_avg_price['Correlation']
                                        + (1-corr_weight)*score_avg_price['Magnitude']
score_avg_price.head(10)

,Store1,Store2,Correlation,Magnitude,Avg Price Score
0,77.0,1.0,-0.799493,0.539282,-0.130106
1,77.0,2.0,-0.514532,0.743444,0.114456
2,77.0,3.0,-0.168485,0.636928,0.234222
3,77.0,4.0,-0.027489,0.593611,0.283061
4,77.0,5.0,-0.045324,0.739516,0.347096
5,77.0,6.0,0.513189,0.657470,0.585330
6,77.0,7.0,-0.304087,0.499314,0.097614
7,77.0,8.0,0.748430,0.465636,0.607033
8,77.0,9.0,0.033687,0.571109,0.302398
9,77.0,10.0,0.565272,0.534494,0.549883


In [80]:
score_sales.set_index(['Store1', 'Store2'], inplace=True)
score_customer.set_index(['Store1', 'Store2'], inplace=True)
score_avg_price.set_index(['Store1', 'Store2'], inplace=True)

score_control = pd.concat([score_sales['Sales Score'], score_customer['Customer Score'],
                           score_avg_price['Avg Price Score']], 
                         axis=1)

score_control.head(15)

Sales Score  Customer Score  Avg Price Score
Store1 Store2                                              
77.0   1.0        0.031106        0.396533        -0.130106
       2.0        0.150041        0.129039         0.114456
       3.0        0.175839        0.173370         0.234222
       4.0        0.019814        0.136960         0.283061
       5.0        0.025691        0.116292         0.347096
       6.0        0.205811        0.425867         0.585330
       7.0        0.136271        0.124546         0.097614
       8.0        0.486094        0.345502         0.607033
       9.0        0.112071        0.126846         0.302398
       10.0       0.104031       -0.074718         0.549883
       12.0       0.108118        0.083492         0.229378
       13.0       0.235170        0.472673         0.276563
       15.0      -0.026031       -0.030218         0.029103
       16.0      -0.039767       -0.060113         0.317294
       17.0       0.462985        0.411187         0.209071

In [81]:
score_control.shape

(239, 3)

In [144]:
score_control['Final'] = 0.5 * (score_control['Sales Score'] + score_control['Customer Score'])

score_control.head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
77.0   1.0        0.031106        0.396533        -0.130106  0.213819
       2.0        0.150041        0.129039         0.114456  0.139540
       3.0        0.175839        0.173370         0.234222  0.174604
       4.0        0.019814        0.136960         0.283061  0.078387
       5.0        0.025691        0.116292         0.347096  0.070992

In [145]:
score_control.sort_values(by='Final', ascending=False).head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
77.0   38.0       0.714492        0.825870         0.339815  0.770181
       187.0      0.767113        0.728812         0.220831  0.747962
       265.0      0.757670        0.680050         0.386710  0.718860
       94.0       0.692070        0.701354         0.583919  0.696712
       88.0       0.743402        0.581507         0.107780  0.662454

We can see that Sore number 38comes out to be the potential control store for 77

In [147]:
control_sore = 38

past_Sales = metric_data.copy()

store_type = []

for i in past_Sales['STORE_NBR']:
    if i == trial_store:
        store_type.append('Trial Store')
    elif i == control_sore:
        store_type.append('Control Store')
    else:
        store_type.append('Other Stores')
        
past_Sales['Store_Type'] = store_type
past_Sales.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Store_Type
0,1,2018-07,77.6,24,22,3.527273,1.090909,3.233333,Other Stores
1,1,2018-08,46.0,15,14,3.285714,1.071429,3.066667,Other Stores
2,1,2018-09,95.7,27,22,4.350000,1.227273,3.544444,Other Stores
3,1,2018-10,81.2,25,17,4.776471,1.470588,3.248000,Other Stores
4,1,2018-11,84.7,27,19,4.457895,1.421053,3.137037,Other Stores


In [128]:
past_Sales['Store_Type'].unique()

array(['Other Stores', 'Trial Store', 'Control Store'], dtype=object)

Let's visualize the total sales for Control, Trial and the mean of other stores for each month

In [148]:
past_Sales['MONTH'] = pd.to_datetime(past_Sales['MONTH_YEAR'])

In [149]:
past_Sales.dtypes

STORE_NBR                            int64
MONTH_YEAR                          object
Total Sales                        float64
Quantity                             int64
Customer_per_store                   int64
Transaction_per_customer           float64
Chips_per_customer                 float64
Averagr Price                      float64
Store_Type                          object
MONTH                       datetime64[ns]
dtype: object

In [150]:
past_Sales.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Store_Type,MONTH
0,1,2018-07,77.6,24,22,3.527273,1.090909,3.233333,Other Stores,2018-07-01
1,1,2018-08,46.0,15,14,3.285714,1.071429,3.066667,Other Stores,2018-08-01
2,1,2018-09,95.7,27,22,4.350000,1.227273,3.544444,Other Stores,2018-09-01
3,1,2018-10,81.2,25,17,4.776471,1.470588,3.248000,Other Stores,2018-10-01
4,1,2018-11,84.7,27,19,4.457895,1.421053,3.137037,Other Stores,2018-11-01


Analyzing total sales 

In [155]:
trial_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Total Sales']]
trial_77.set_index('MONTH', inplace=True)
trial_77.rename(columns={'Total Sales':'Trial Store'}, inplace=True)

control_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Total Sales']]
control_77.set_index('MONTH', inplace=True)
control_77.rename(columns={'Total Sales':'Control Store'}, inplace=True)

others_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Total Sales']]
others_77 = pd.DataFrame(others_77.groupby('MONTH')['Total Sales'].mean())
others_77.rename(columns={'Total Sales':'Other Stores'}, inplace=True)

combine_sales = pd.concat([control_77, trial_77, others_77], axis=1)

combine_sales.to_csv('Combines_Sales_77.csv', index=True)

Analyzing Total Customers

In [160]:
trial_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Customer_per_store']]
trial_77.set_index('MONTH', inplace=True)
trial_77.rename(columns={'Customer_per_store':'Trial Store'}, inplace=True)

control_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Customer_per_store']]
control_77.set_index('MONTH', inplace=True)
control_77.rename(columns={'Customer_per_store':'Control Store'}, inplace=True)

others_77 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Customer_per_store']]
others_77 = pd.DataFrame(others_77.groupby('MONTH')['Customer_per_store'].mean())
others_77.rename(columns={'Customer_per_store':'Other Stores'}, inplace=True)

combine_customer = pd.concat([control_77, trial_77, others_77], axis=1)

combine_customer.to_csv('Combines_Cusotmers_77.csv', index=True)

In [161]:
combine_customer

,Control Store,Trial Store,Other Stores
MONTH,,,
2018-07-01,18,19,30.168776
2018-08-01,16,17,29.071730
2018-09-01,11,12,28.607595
2018-10-01,13,11,29.649789
2018-11-01,21,18,28.654008
2018-12-01,13,14,30.016878
2019-01-01,15,14,29.265823


Assessment fot trial store 77

In [163]:
metric_data.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
0,1,2018-07,77.6,24,22,3.527273,1.090909,3.233333
1,1,2018-08,46.0,15,14,3.285714,1.071429,3.066667
2,1,2018-09,95.7,27,22,4.350000,1.227273,3.544444
3,1,2018-10,81.2,25,17,4.776471,1.470588,3.248000
4,1,2018-11,84.7,27,19,4.457895,1.421053,3.137037


In [164]:
past_Sales.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Store_Type,MONTH
0,1,2018-07,77.6,24,22,3.527273,1.090909,3.233333,Other Stores,2018-07-01
1,1,2018-08,46.0,15,14,3.285714,1.071429,3.066667,Other Stores,2018-08-01
2,1,2018-09,95.7,27,22,4.350000,1.227273,3.544444,Other Stores,2018-09-01
3,1,2018-10,81.2,25,17,4.776471,1.470588,3.248000,Other Stores,2018-10-01
4,1,2018-11,84.7,27,19,4.457895,1.421053,3.137037,Other Stores,2018-11-01


In [166]:
trial_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Total Sales'].sum()
control_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Total Sales'].sum()
scaling_sales = trial_sum/control_sum
scaling_sales

0.9767640526225868

In [170]:
scaled_control_stores = metric.copy()

scaled_control_stores = scaled_control_stores.loc[scaled_control_stores['STORE_NBR'] == control_sore]


scaled_control_stores['Control Sales'] = scaled_control_stores['Total Sales']*scaling_sales
scaled_control_stores.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000
413,38,2018-12,81.4,23,13,6.261538,1.769231,3.539130
414,38,2019-01,71.4,23,15,4.760000,1.533333,3.104348
415,38,2019-02,81.8,25,16,5.112500,1.562500,3.272000
416,38,2019-03,64.6,20,12,5.383333,1.666667,3.230000
417,38,2019-04,74.9,22,13,5.761538,1.692308,3.404545


In [172]:
percent_off = scaled_control_stores[['MONTH_YEAR', 'Control Sales']]
percent_off.reset_index(drop=True, inplace=True)

trial_sales = metric.loc[metric['STORE_NBR'] == trial_store, 'Total Sales']
trial_sales.reset_index(drop=True, inplace=True)

percent_off = pd.concat([percent_off, trial_sales], axis=1)
percent_off.rename(columns={'Total Sales':'Trial Sales'}, inplace=True)

percent_off['Percentage_diff'] = abs(percent_off['Control Sales'] - percent_off['Trial Sales'])/percent_off['Control Sales']
percent_off

,MONTH_YEAR,Control Sales,Trial Sales
0,2018-07,98.262464,105.3
1,2018-08,77.066684,90.4
2,2018-09,63.196634,52.5
3,2018-10,77.652742,47.4
4,2018-11,106.271929,103.0
5,2018-12,79.508594,87.7
6,2019-01,69.740953,85.4
7,2019-02,79.899300,75.8
8,2019-03,63.098958,72.2
9,2019-04,73.159628,109.8


Our null hypothesis is such that out trial period is same as the pre-trial period

In [174]:
from statistics import stdev

In [176]:
std_dev = stdev(percent_off.loc[percent_off['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])
std_dev

0.11876720305646335

In [177]:
dof = 6

In [178]:
percent_off['tvalue'] = percent_off['Percentage_diff']/std_dev
percent_off.loc[(percent_off['MONTH_YEAR'] > '2019-01') & (percent_off['MONTH_YEAR'] < '2019-05'), 'tvalue']  

7    0.431986
8    1.214430
9    4.216886
Name: tvalue, dtype: float64

In [180]:
from scipy.stats import t

In [181]:
# Finding the 95th percentile for T-distribution with dof = 6

t.isf(0.05, dof)

1.943180280392782

In [182]:
scaled_control_stores.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Sales
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926,98.262464
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435,77.066684
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263,63.196634
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000,77.652742
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000,106.271929


In [183]:
scaled_control_stores['MONTH'] = pd.to_datetime(scaled_control_stores['MONTH_YEAR'])
scaled_control_stores

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Sales,MONTH
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926,98.262464,2018-07-01
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435,77.066684,2018-08-01
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263,63.196634,2018-09-01
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000,77.652742,2018-10-01
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000,106.271929,2018-11-01
413,38,2018-12,81.4,23,13,6.261538,1.769231,3.539130,79.508594,2018-12-01
414,38,2019-01,71.4,23,15,4.760000,1.533333,3.104348,69.740953,2019-01-01
415,38,2019-02,81.8,25,16,5.112500,1.562500,3.272000,79.899300,2019-02-01
416,38,2019-03,64.6,20,12,5.383333,1.666667,3.230000,63.098958,2019-03-01
417,38,2019-04,74.9,22,13,5.761538,1.692308,3.404545,73.159628,2019-04-01


In [184]:
controlSales = scaled_control_stores.loc[:, ['MONTH', 'Control Sales']]
controlSales.set_index('MONTH', inplace=True)

# Let's calculate the 5% and 95% confidence interval
controlSales['Control 5% Confidence Interval'] = controlSales['Control Sales'] * (1 - std_dev*2)
controlSales['Control 95% Confidence Interval'] = controlSales['Control Sales'] * (1 + std_dev*2)
controlSales

,Control Sales
MONTH,
2018-07-01,98.262464
2018-08-01,77.066684
2018-09-01,63.196634
2018-10-01,77.652742
2018-11-01,106.271929
2018-12-01,79.508594
2019-01-01,69.740953
2019-02-01,79.899300
2019-03-01,63.098958


In [205]:
Combined_77 = pd.merge(controlSales, trial_Sales, right_index=True, left_index=True)
Combined_77

,Control Sales,Control 5% Confidence Interval,Control 95% Confidence Interval,Trial Sales
MONTH,,,,
2018-07-01,98.262464,74.921748,121.603180,105.3
2018-08-01,77.066684,58.760695,95.372673,90.4
2018-09-01,63.196634,48.185259,78.208009,52.5
2018-10-01,77.652742,59.207544,96.097940,47.4
2018-11-01,106.271929,81.028689,131.515168,103.0
2018-12-01,79.508594,60.622567,98.394621,87.7
2019-01-01,69.740953,53.175077,86.306829,85.4
2019-02-01,79.899300,60.920467,98.878132,75.8
2019-03-01,63.098958,48.110784,78.087131,72.2


In [203]:
trial_Sales = percent_off[['MONTH_YEAR', 'Trial Sales']]
trial_Sales['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trial_Sales.set_index('MONTH', inplace=True)

trial_Sales.drop(['MONTH_YEAR'], axis=1, inplace=True)

C:\Users\LENOVO\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [206]:
Combined_77.to_csv('Test_77.csv', index=True)

Analyzing assessment for Customers for Store 77

In [207]:
trial_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Customer_per_store'].sum()
control_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Customer_per_store'].sum()
scaling_customers = trial_customers/control_customers
scaling_customers

0.9813084112149533

In [211]:
scaled_control_customers = metric.copy()

scaled_control_customers = scaled_control_customers.loc[scaled_control_customers['STORE_NBR'] == control_sore]
scaled_control_customers

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000
413,38,2018-12,81.4,23,13,6.261538,1.769231,3.539130
414,38,2019-01,71.4,23,15,4.760000,1.533333,3.104348
415,38,2019-02,81.8,25,16,5.112500,1.562500,3.272000
416,38,2019-03,64.6,20,12,5.383333,1.666667,3.230000
417,38,2019-04,74.9,22,13,5.761538,1.692308,3.404545


In [212]:
scaled_control_customers['Control Customers'] = scaled_control_stores['Customer_per_store']*scaling_customers
scaled_control_customers.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Customers
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926,17.663551
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435,15.700935
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263,10.794393
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000,12.757009
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000,20.607477


In [215]:
percent_offc = scaled_control_customers[['MONTH_YEAR', 'Control Customers']]
percent_offc.reset_index(drop=True, inplace=True)

trial_customers = metric.loc[metric['STORE_NBR'] == trial_store, 'Customer_per_store']
trial_customers.reset_index(drop=True, inplace=True)

percent_offc = pd.concat([percent_offc, trial_customers], axis=1)
percent_offc.rename(columns={'Customer_per_store':'Trial Customers'}, inplace=True)

percent_offc

,MONTH_YEAR,Control Customers,Trial Customers
0,2018-07,17.663551,19
1,2018-08,15.700935,17
2,2018-09,10.794393,12
3,2018-10,12.757009,11
4,2018-11,20.607477,18
5,2018-12,12.757009,14
6,2019-01,14.719626,14
7,2019-02,15.700935,16
8,2019-03,11.775701,15
9,2019-04,12.757009,18


In [216]:
percent_offc['Percentage_diff'] = abs(percent_offc['Control Customers'] - 
                                      percent_offc['Trial Customers'])/percent_offc['Control Customers']
percent_offc

,MONTH_YEAR,Control Customers,Trial Customers,Percentage_diff
0,2018-07,17.663551,19,0.075661
1,2018-08,15.700935,17,0.082738
2,2018-09,10.794393,12,0.111688
3,2018-10,12.757009,11,0.137729
4,2018-11,20.607477,18,0.126531
5,2018-12,12.757009,14,0.097436
6,2019-01,14.719626,14,0.048889
7,2019-02,15.700935,16,0.019048
8,2019-03,11.775701,15,0.273810
9,2019-04,12.757009,18,0.410989


In [217]:
std_devc = stdev(percent_offc.loc[percent_offc['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])
std_devc

0.030875714127836965

In [218]:
percent_offc['tvalue'] = percent_offc['Percentage_diff']/std_devc
percent_offc.loc[(percent_offc['MONTH_YEAR'] > '2019-01') & (percent_offc['MONTH_YEAR'] < '2019-05'), 'tvalue']  

7     0.616913
8     8.868120
9    13.311077
Name: tvalue, dtype: float64

In [219]:
t.isf(0.05, dof)

1.943180280392782

In [220]:
scaled_control_customers

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Customers
408,38,2018-07,100.6,27,18,5.588889,1.500000,3.725926,17.663551
409,38,2018-08,78.9,23,16,4.931250,1.437500,3.430435,15.700935
410,38,2018-09,64.7,19,11,5.881818,1.727273,3.405263,10.794393
411,38,2018-10,79.5,20,13,6.115385,1.538462,3.975000,12.757009
412,38,2018-11,108.8,34,21,5.180952,1.619048,3.200000,20.607477
413,38,2018-12,81.4,23,13,6.261538,1.769231,3.539130,12.757009
414,38,2019-01,71.4,23,15,4.760000,1.533333,3.104348,14.719626
415,38,2019-02,81.8,25,16,5.112500,1.562500,3.272000,15.700935
416,38,2019-03,64.6,20,12,5.383333,1.666667,3.230000,11.775701
417,38,2019-04,74.9,22,13,5.761538,1.692308,3.404545,12.757009


In [222]:
scaled_control_customers['MONTH'] = pd.to_datetime(scaled_control_customers['MONTH_YEAR'])

controlCustomers= scaled_control_customers.loc[:, ['MONTH', 'Control Customers']]
controlCustomers.set_index('MONTH', inplace=True)

In [223]:
controlCustomers

,Control Customers
MONTH,
2018-07-01,17.663551
2018-08-01,15.700935
2018-09-01,10.794393
2018-10-01,12.757009
2018-11-01,20.607477
2018-12-01,12.757009
2019-01-01,14.719626
2019-02-01,15.700935
2019-03-01,11.775701


In [225]:
# Let's calculate the 5% and 95% confidence interval

controlCustomers['Control 5% Confidence Interval'] = controlCustomers['Control Customers'] * (1 - std_devc*2)
controlCustomers['Control 95% Confidence Interval'] = controlCustomers['Control Customers'] * (1 + std_devc*2)
controlCustomers

,Control Customers,Control 5% Confidence Interval,Control 95% Confidence Interval
MONTH,,,
2018-07-01,17.663551,16.572802,18.754301
2018-08-01,15.700935,14.731379,16.670490
2018-09-01,10.794393,10.127823,11.460962
2018-10-01,12.757009,11.969246,13.544773
2018-11-01,20.607477,19.334936,21.880018
2018-12-01,12.757009,11.969246,13.544773
2019-01-01,14.719626,13.810668,15.628584
2019-02-01,15.700935,14.731379,16.670490
2019-03-01,11.775701,11.048535,12.502867


In [229]:
Combined_77c = pd.merge(controlCustomers, trialCustomers, right_index=True, left_index=True)
Combined_77c

,Control Customers,Control 5% Confidence Interval,Control 95% Confidence Interval,Trial Customers
MONTH,,,,
2018-07-01,17.663551,16.572802,18.754301,19
2018-08-01,15.700935,14.731379,16.670490,17
2018-09-01,10.794393,10.127823,11.460962,12
2018-10-01,12.757009,11.969246,13.544773,11
2018-11-01,20.607477,19.334936,21.880018,18
2018-12-01,12.757009,11.969246,13.544773,14
2019-01-01,14.719626,13.810668,15.628584,14
2019-02-01,15.700935,14.731379,16.670490,16
2019-03-01,11.775701,11.048535,12.502867,15


In [227]:
trialCustomers = percent_offc[['MONTH_YEAR', 'Trial Customers']]
trialCustomers['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trialCustomers.set_index('MONTH', inplace=True)
trialCustomers.drop(['MONTH_YEAR'], axis=1, inplace=True)

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\LENOVO\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [228]:
trialCustomers

,Trial Customers
MONTH,
2018-07-01,19
2018-08-01,17
2018-09-01,12
2018-10-01,11
2018-11-01,18
2018-12-01,14
2019-01-01,14
2019-02-01,16
2019-03-01,15


In [230]:
Combined_77c.to_csv('Customer_77.csv', index=True)

Let's find the control store for trial store 86

In [231]:
trial_store = 86

corr_sales = calcorr(metric_data, 'Total Sales', trial_store)
corr_customer = calcorr(metric_data, 'Customer_per_store', trial_store)
corr_avg_price = calcorr(metric_data, 'Averagr Price', trial_store)

mag_sales = caldist(metric_data, 'Total Sales', trial_store)
mag_customer = caldist(metric_data, 'Customer_per_store', trial_store)
mag_avg_price = caldist(metric_data, 'Averagr Price', trial_store)

score_sales = pd.concat([corr_sales, mag_sales['Magnitude']], axis=1)

# Adding an additional column in 'score_sales' which calculates the weighted average

corr_weight = 0.5

score_sales['Sales Score'] = corr_weight*score_sales['Correlation'] + (1-corr_weight)*score_sales['Magnitude']
score_sales.head(10)

,Store1,Store2,Correlation,Magnitude,Sales Score
0,86.0,1.0,-0.083409,0.476207,0.196399
1,86.0,2.0,-0.216248,0.554284,0.169018
2,86.0,3.0,-0.347220,0.475638,0.064209
3,86.0,4.0,-0.625945,0.442024,-0.091960
4,86.0,5.0,0.351626,0.650317,0.500972
5,86.0,6.0,0.328423,0.466740,0.397582
6,86.0,7.0,0.442095,0.563571,0.502833
7,86.0,8.0,0.066514,0.597615,0.332065
8,86.0,9.0,0.789559,0.616734,0.703147
9,86.0,10.0,-0.416070,0.607081,0.095506


In [232]:
score_customer = pd.concat([corr_customer, mag_customer['Magnitude']], axis=1)

score_customer['Customer Score'] = corr_weight*score_customer['Correlation'] + (1-corr_weight)*score_customer['Magnitude']
score_customer.head(10)

,Store1,Store2,Correlation,Magnitude,Customer Score
0,86.0,1.0,-0.456823,0.434066,-0.011379
1,86.0,2.0,0.102156,0.547619,0.324887
2,86.0,3.0,-0.152344,0.672269,0.259962
3,86.0,4.0,-0.859210,0.563492,-0.147859
4,86.0,5.0,0.345018,0.589286,0.467152
5,86.0,6.0,0.142420,0.473684,0.308052
6,86.0,7.0,0.648242,0.609524,0.628883
7,86.0,8.0,0.113406,0.591837,0.352622
8,86.0,9.0,0.560547,0.527473,0.544010
9,86.0,10.0,-0.026643,0.771429,0.372393


In [233]:
score_avg_price = pd.concat([corr_avg_price, mag_avg_price['Magnitude']], axis=1)

score_avg_price['Avg Price Score'] = corr_weight*score_avg_price['Correlation'] + (1-corr_weight)*score_avg_price['Magnitude']
score_avg_price.head(10)

,Store1,Store2,Correlation,Magnitude,Avg Price Score
0,86.0,1.0,-0.423647,0.599211,0.087782
1,86.0,2.0,0.450593,0.693962,0.572277
2,86.0,3.0,-0.465455,0.309959,-0.077748
3,86.0,4.0,-0.440009,0.629520,0.094755
4,86.0,5.0,-0.331590,0.491458,0.079934
5,86.0,6.0,-0.427151,0.640308,0.106578
6,86.0,7.0,0.461110,0.601167,0.531139
7,86.0,8.0,0.092357,0.460476,0.276417
8,86.0,9.0,0.020304,0.660093,0.340199
9,86.0,10.0,0.569097,0.611653,0.590375


In [234]:
score_sales.set_index(['Store1', 'Store2'], inplace=True)
score_customer.set_index(['Store1', 'Store2'], inplace=True)
score_avg_price.set_index(['Store1', 'Store2'], inplace=True)

score_control = pd.concat([score_sales['Sales Score'], score_customer['Customer Score'], score_avg_price['Avg Price Score']], 
                         axis=1)

score_control.head(15)

Sales Score  Customer Score  Avg Price Score
Store1 Store2                                              
86.0   1.0        0.196399       -0.011379         0.087782
       2.0        0.169018        0.324887         0.572277
       3.0        0.064209        0.259962        -0.077748
       4.0       -0.091960       -0.147859         0.094755
       5.0        0.500972        0.467152         0.079934
       6.0        0.397582        0.308052         0.106578
       7.0        0.502833        0.628883         0.531139
       8.0        0.332065        0.352622         0.276417
       9.0        0.703147        0.544010         0.340199
       10.0       0.095506        0.372393         0.590375
       12.0       0.412397        0.325190         0.058596
       13.0       0.548054        0.500760        -0.123802
       15.0       0.428956        0.435593         0.149897
       16.0       0.269603        0.557514         0.647378
       17.0       0.032724       -0.053997        -0.093824

In [242]:
score_control['Final'] = 0.5 * (score_control['Sales Score'] + score_control['Customer Score']) #+ score_control['Avg Price Score'])

score_control.head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
86.0   1.0        0.196399       -0.011379         0.087782  0.092510
       2.0        0.169018        0.324887         0.572277  0.246953
       3.0        0.064209        0.259962        -0.077748  0.162086
       4.0       -0.091960       -0.147859         0.094755 -0.119909
       5.0        0.500972        0.467152         0.079934  0.484062

In [243]:
score_control.sort_values(by='Final', ascending=False).head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
86.0   126.0      0.669425        0.660669         0.078560  0.665047
       110.0      0.623787        0.678269         0.502443  0.651028
       172.0      0.651693        0.603671        -0.029195  0.627682
       9.0        0.703147        0.544010         0.340199  0.623578
       186.0      0.601502        0.620454         0.669423  0.610978

We can see that the control store for Store 86 comes out to be 155

In [253]:
control_sore = 155

past_Sales = metric_data.copy()

store_type = []

for i in past_Sales['STORE_NBR']:
    if i == trial_store:
        store_type.append('Trial Store')
    elif i == control_sore:
        store_type.append('Control Store')
    else:
        store_type.append('Other Stores')
        
past_Sales['Store_Type'] = store_type

past_Sales['MONTH'] = pd.to_datetime(past_Sales['MONTH_YEAR'])

trial_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Total Sales']]
trial_86.set_index('MONTH', inplace=True)
trial_86.rename(columns={'Total Sales':'Trial Store'}, inplace=True)

control_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Total Sales']]
control_86.set_index('MONTH', inplace=True)
control_86.rename(columns={'Total Sales':'Control Store'}, inplace=True)

others_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Total Sales']]
others_86 = pd.DataFrame(others_86.groupby('MONTH')['Total Sales'].mean())
others_86.rename(columns={'Total Sales':'Other Stores'}, inplace=True)

combine_sales = pd.concat([control_86, trial_86, others_86], axis=1)

combine_sales.to_csv('Combines_Sales_86.csv', index=True)

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Store_Type
0,1,2018-07,77.6,24,22,3.527273,1.090909,3.233333,Other Stores
1,1,2018-08,46.0,15,14,3.285714,1.071429,3.066667,Other Stores
2,1,2018-09,95.7,27,22,4.350000,1.227273,3.544444,Other Stores
3,1,2018-10,81.2,25,17,4.776471,1.470588,3.248000,Other Stores
4,1,2018-11,84.7,27,19,4.457895,1.421053,3.137037,Other Stores


In [256]:
trial_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Customer_per_store']]
trial_86.set_index('MONTH', inplace=True)
trial_86.rename(columns={'Customer_per_store':'Trial Store'}, inplace=True)

control_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Customer_per_store']]
control_86.set_index('MONTH', inplace=True)
control_86.rename(columns={'Customer_per_store':'Control Store'}, inplace=True)

others_86 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Customer_per_store']]
others_86 = pd.DataFrame(others_86.groupby('MONTH')['Customer_per_store'].mean())
others_86.rename(columns={'Customer_per_store':'Other Stores'}, inplace=True)

combine_customer = pd.concat([control_86, trial_86, others_86], axis=1)

combine_customer.to_csv('Combines_Cusotmers_86.csv', index=True)

Assessment fot Store 86

In [257]:
trial_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Total Sales'].sum()
control_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Total Sales'].sum()
scaling_sales = trial_sum/control_sum
scaling_sales

1.1109004499721926

In [258]:
scaled_control_stores = metric.copy()

scaled_control_stores = scaled_control_stores.loc[scaled_control_stores['STORE_NBR'] == control_sore]
scaled_control_stores

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
1656,155,2018-07,256.60,72,33,7.775758,2.181818,3.563889
1657,155,2018-08,253.10,70,33,7.669697,2.121212,3.615714
1658,155,2018-09,306.40,88,42,7.295238,2.095238,3.481818
1659,155,2018-10,316.80,88,41,7.726829,2.146341,3.600000
1660,155,2018-11,311.60,92,41,7.600000,2.243902,3.386957
1661,155,2018-12,240.60,66,32,7.518750,2.062500,3.645455
1662,155,2019-01,292.80,78,37,7.913514,2.108108,3.753846
1663,155,2019-02,325.60,92,41,7.941463,2.243902,3.539130
1664,155,2019-03,241.40,80,37,6.524324,2.162162,3.017500
1665,155,2019-04,288.40,84,38,7.589474,2.210526,3.433333


In [259]:
scaled_control_stores['Control Sales'] = scaled_control_stores['Total Sales']*scaling_sales


percent_off = scaled_control_stores[['MONTH_YEAR', 'Control Sales']]
percent_off.reset_index(drop=True, inplace=True)

trial_sales = metric.loc[metric['STORE_NBR'] == trial_store, 'Total Sales']
trial_sales.reset_index(drop=True, inplace=True)

percent_off = pd.concat([percent_off, trial_sales], axis=1)
percent_off.rename(columns={'Total Sales':'Trial Sales'}, inplace=True)

percent_off

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Sales
1656,155,2018-07,256.6,72,33,7.775758,2.181818,3.563889,285.057055
1657,155,2018-08,253.1,70,33,7.669697,2.121212,3.615714,281.168904
1658,155,2018-09,306.4,88,42,7.295238,2.095238,3.481818,340.379898
1659,155,2018-10,316.8,88,41,7.726829,2.146341,3.600000,351.933263
1660,155,2018-11,311.6,92,41,7.600000,2.243902,3.386957,346.156580


In [265]:
percent_off['Percentage_diff'] = abs(percent_off['Control Sales'] - percent_off['Trial Sales'])/percent_off['Control Sales']


std_dev = stdev(percent_off.loc[percent_off['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])


percent_off['tvalue'] = percent_off['Percentage_diff']/std_dev
percent_off.loc[(percent_off['MONTH_YEAR'] > '2019-01') & (percent_off['MONTH_YEAR'] < '2019-05'), 'tvalue']  

7    0.544044
8    4.180504
9    0.185545
Name: tvalue, dtype: float64

In [266]:
t.isf(0.05, dof)

1.943180280392782

In [267]:
scaled_control_stores['MONTH'] = pd.to_datetime(scaled_control_stores['MONTH_YEAR'])
scaled_control_stores

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Sales,MONTH
1656,155,2018-07,256.60,72,33,7.775758,2.181818,3.563889,285.057055,2018-07-01
1657,155,2018-08,253.10,70,33,7.669697,2.121212,3.615714,281.168904,2018-08-01
1658,155,2018-09,306.40,88,42,7.295238,2.095238,3.481818,340.379898,2018-09-01
1659,155,2018-10,316.80,88,41,7.726829,2.146341,3.600000,351.933263,2018-10-01
1660,155,2018-11,311.60,92,41,7.600000,2.243902,3.386957,346.156580,2018-11-01
1661,155,2018-12,240.60,66,32,7.518750,2.062500,3.645455,267.282648,2018-12-01
1662,155,2019-01,292.80,78,37,7.913514,2.108108,3.753846,325.271652,2019-01-01
1663,155,2019-02,325.60,92,41,7.941463,2.243902,3.539130,361.709187,2019-02-01
1664,155,2019-03,241.40,80,37,6.524324,2.162162,3.017500,268.171369,2019-03-01
1665,155,2019-04,288.40,84,38,7.589474,2.210526,3.433333,320.383690,2019-04-01


In [268]:
controlSales = scaled_control_stores.loc[:, ['MONTH', 'Control Sales']]
controlSales.set_index('MONTH', inplace=True)
controlSales

,Control Sales
MONTH,
2018-07-01,285.057055
2018-08-01,281.168904
2018-09-01,340.379898
2018-10-01,351.933263
2018-11-01,346.156580
2018-12-01,267.282648
2019-01-01,325.271652
2019-02-01,361.709187
2019-03-01,268.171369


In [269]:
# Let's calculate the 5% and 95% confidence interval

controlSales['Control 5% Confidence Interval'] = controlSales['Control Sales'] * (1 - std_dev*2)
controlSales['Control 95% Confidence Interval'] = controlSales['Control Sales'] * (1 + std_dev*2)


trial_Sales = percent_off[['MONTH_YEAR', 'Trial Sales']]
trial_Sales['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trial_Sales.set_index('MONTH', inplace=True)

trial_Sales.drop(['MONTH_YEAR'], axis=1, inplace=True)

,Control Sales,Control 5% Confidence Interval,Control 95% Confidence Interval
MONTH,,,
2018-07-01,285.057055,252.292847,317.821264
2018-08-01,281.168904,248.851596,313.486212
2018-09-01,340.379898,301.256930,379.502866
2018-10-01,351.933263,311.482361,392.384164
2018-11-01,346.156580,306.369646,385.943515
2018-12-01,267.282648,236.561415,298.003882
2019-01-01,325.271652,287.885213,362.658091
2019-02-01,361.709187,320.134649,403.283724
2019-03-01,268.171369,237.347986,298.994751


In [272]:
Combined_86 = pd.merge(controlSales, trial_Sales, right_index=True, left_index=True)


In [274]:
Combined_86.to_csv('Test_86.csv', index=True)

In [273]:
Combined_86

,Control Sales,Control 5% Confidence Interval,Control 95% Confidence Interval,Trial Sales
MONTH,,,,
2018-07-01,285.057055,252.292847,317.821264,251.00
2018-08-01,281.168904,248.851596,313.486212,274.05
2018-09-01,340.379898,301.256930,379.502866,301.00
2018-10-01,351.933263,311.482361,392.384164,304.40
2018-11-01,346.156580,306.369646,385.943515,388.20
2018-12-01,267.282648,236.561415,298.003882,325.40
2019-01-01,325.271652,287.885213,362.658091,353.20
2019-02-01,361.709187,320.134649,403.283724,350.40
2019-03-01,268.171369,237.347986,298.994751,332.60


In [275]:
trial_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Customer_per_store'].sum()
control_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Customer_per_store'].sum()
scaling_customers = trial_customers/control_customers
scaling_customers

1.1544401544401544

In [276]:
scaled_control_customers = metric.copy()

scaled_control_customers = scaled_control_customers.loc[scaled_control_customers['STORE_NBR'] == control_sore]


scaled_control_customers['Control Customers'] = scaled_control_stores['Customer_per_store']*scaling_customers
scaled_control_customers.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
1656,155,2018-07,256.60,72,33,7.775758,2.181818,3.563889
1657,155,2018-08,253.10,70,33,7.669697,2.121212,3.615714
1658,155,2018-09,306.40,88,42,7.295238,2.095238,3.481818
1659,155,2018-10,316.80,88,41,7.726829,2.146341,3.600000
1660,155,2018-11,311.60,92,41,7.600000,2.243902,3.386957
1661,155,2018-12,240.60,66,32,7.518750,2.062500,3.645455
1662,155,2019-01,292.80,78,37,7.913514,2.108108,3.753846
1663,155,2019-02,325.60,92,41,7.941463,2.243902,3.539130
1664,155,2019-03,241.40,80,37,6.524324,2.162162,3.017500
1665,155,2019-04,288.40,84,38,7.589474,2.210526,3.433333


In [278]:
percent_offc = scaled_control_customers[['MONTH_YEAR', 'Control Customers']]
percent_offc.reset_index(drop=True, inplace=True)

trial_customers = metric.loc[metric['STORE_NBR'] == trial_store, 'Customer_per_store']
trial_customers.reset_index(drop=True, inplace=True)

percent_offc = pd.concat([percent_offc, trial_customers], axis=1)
percent_offc.rename(columns={'Customer_per_store':'Trial Customers'}, inplace=True)

percent_offc

,MONTH_YEAR,Control Customers,Trial Customers
0,2018-07,38.096525,34
1,2018-08,38.096525,38
2,2018-09,48.486486,39
3,2018-10,47.332046,42
4,2018-11,47.332046,52
5,2018-12,36.942085,48
6,2019-01,42.714286,46
7,2019-02,47.332046,46
8,2019-03,42.714286,42
9,2019-04,43.868726,45


In [279]:
percent_offc['Percentage_diff'] = abs(percent_offc['Control Customers'] - 
                                      percent_offc['Trial Customers'])/percent_offc['Control Customers']
percent_offc

,MONTH_YEAR,Control Customers,Trial Customers,Percentage_diff
0,2018-07,38.096525,34,0.107530
1,2018-08,38.096525,38,0.002534
2,2018-09,48.486486,39,0.195652
3,2018-10,47.332046,42,0.112652
4,2018-11,47.332046,52,0.098621
5,2018-12,36.942085,48,0.299331
6,2019-01,42.714286,46,0.076923
7,2019-02,47.332046,46,0.028143
8,2019-03,42.714286,42,0.016722
9,2019-04,43.868726,45,0.025788


In [280]:
std_devc = stdev(percent_offc.loc[percent_offc['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])


percent_offc['tvalue'] = percent_offc['Percentage_diff']/std_devc
percent_offc.loc[(percent_offc['MONTH_YEAR'] > '2019-01') & (percent_offc['MONTH_YEAR'] < '2019-05'), 'tvalue']  

scaled_control_customers['MONTH'] = pd.to_datetime(scaled_control_customers['MONTH_YEAR'])

controlCustomers= scaled_control_customers.loc[:, ['MONTH', 'Control Customers']]
controlCustomers.set_index('MONTH', inplace=True)

0.09469107856293658

In [283]:
controlCustomers

,Control Customers
MONTH,
2018-07-01,38.096525
2018-08-01,38.096525
2018-09-01,48.486486
2018-10-01,47.332046
2018-11-01,47.332046
2018-12-01,36.942085
2019-01-01,42.714286
2019-02-01,47.332046
2019-03-01,42.714286


In [284]:
# Let's calculate the 5% and 95% confidence interval

controlCustomers['Control 5% Confidence Interval'] = controlCustomers['Control Customers'] * (1 - std_devc*2)
controlCustomers['Control 95% Confidence Interval'] = controlCustomers['Control Customers'] * (1 + std_devc*2)
controlCustomers

,Control Customers,Control 5% Confidence Interval,Control 95% Confidence Interval
MONTH,,,
2018-07-01,38.096525,30.881723,45.311327
2018-08-01,38.096525,30.881723,45.311327
2018-09-01,48.486486,39.304011,57.668962
2018-10-01,47.332046,38.368201,56.295891
2018-11-01,47.332046,38.368201,56.295891
2018-12-01,36.942085,29.945913,43.938257
2019-01-01,42.714286,34.624962,50.803609
2019-02-01,47.332046,38.368201,56.295891
2019-03-01,42.714286,34.624962,50.803609


In [285]:
trialCustomers = percent_offc[['MONTH_YEAR', 'Trial Customers']]
trialCustomers['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trialCustomers.set_index('MONTH', inplace=True)
trialCustomers.drop(['MONTH_YEAR'], axis=1, inplace=True)

Combined_86c = pd.merge(controlCustomers, trialCustomers, right_index=True, left_index=True)
Combined_86c

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\LENOVO\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [287]:
Combined_86c.to_csv('Customer_86.csv', index=True)

Working for Store 88

In [288]:
trial_store = 88

corr_sales = calcorr(metric_data, 'Total Sales', trial_store)
corr_customer = calcorr(metric_data, 'Customer_per_store', trial_store)
corr_avg_price = calcorr(metric_data, 'Averagr Price', trial_store)

mag_sales = caldist(metric_data, 'Total Sales', trial_store)
mag_customer = caldist(metric_data, 'Customer_per_store', trial_store)
mag_avg_price = caldist(metric_data, 'Averagr Price', trial_store)

score_sales = pd.concat([corr_sales, mag_sales['Magnitude']], axis=1)

# Adding an additional column in 'score_sales' which calculates the weighted average

corr_weight = 0.5

score_sales['Sales Score'] = corr_weight*score_sales['Correlation'] + (1-corr_weight)*score_sales['Magnitude']
score_sales.head(10)

,Store1,Store2,Correlation,Magnitude,Sales Score
0,88.0,1.0,-0.572133,0.554333,-0.008900
1,88.0,2.0,-0.516983,0.564708,0.023862
2,88.0,3.0,0.319839,0.558399,0.439119
3,88.0,4.0,-0.567275,0.590110,0.011417
4,88.0,5.0,-0.343602,0.355689,0.006044
5,88.0,6.0,-0.226224,0.540421,0.157099
6,88.0,7.0,-0.532369,0.590529,0.029080
7,88.0,8.0,0.648997,0.487302,0.568149
8,88.0,9.0,-0.307994,0.512298,0.102152
9,88.0,10.0,0.114945,0.474280,0.294613


In [289]:
score_customer = pd.concat([corr_customer, mag_customer['Magnitude']], axis=1)

score_customer['Customer Score'] = corr_weight*score_customer['Correlation'] + (1-corr_weight)*score_customer['Magnitude']
score_customer.head(10)

,Store1,Store2,Correlation,Magnitude,Customer Score
0,88.0,1.0,-0.413903,0.491071,0.038584
1,88.0,2.0,-0.306362,0.571429,0.132533
2,88.0,3.0,0.328331,0.530612,0.429472
3,88.0,4.0,-0.670298,0.580357,-0.044971
4,88.0,5.0,-0.338510,0.400000,0.030745
5,88.0,6.0,-0.236803,0.516484,0.139840
6,88.0,7.0,-0.217480,0.616883,0.199702
7,88.0,8.0,0.571268,0.532468,0.551868
8,88.0,9.0,-0.255739,0.470238,0.107250
9,88.0,10.0,0.172512,0.660714,0.416613


In [291]:
score_avg_price = pd.concat([corr_avg_price, mag_avg_price['Magnitude']], axis=1)

score_avg_price['Avg Price Score'] = corr_weight*score_avg_price['Correlation'] + (1-corr_weight)*score_avg_price['Magnitude']
score_avg_price.head(10)

,Store1,Store2,Correlation,Magnitude,Avg Price Score
0,88.0,1.0,0.042708,0.395432,0.219070
1,88.0,2.0,0.564723,0.537234,0.550978
2,88.0,3.0,-0.162430,0.556078,0.196824
3,88.0,4.0,-0.226891,0.660998,0.217053
4,88.0,5.0,-0.178980,0.350134,0.085577
5,88.0,6.0,0.102994,0.389664,0.246329
6,88.0,7.0,-0.338622,0.567769,0.114574
7,88.0,8.0,-0.326120,0.467174,0.070527
8,88.0,9.0,0.126647,0.381683,0.254165
9,88.0,10.0,-0.472438,0.640270,0.083916


In [298]:
score_sales.set_index(['Store1', 'Store2'], inplace=True)
score_customer.set_index(['Store1', 'Store2'], inplace=True)
score_avg_price.set_index(['Store1', 'Store2'], inplace=True)

In [299]:
score_control = pd.concat([score_sales['Sales Score'], score_customer['Customer Score'], score_avg_price['Avg Price Score']], 
                         axis=1)

score_control.head(15)

Sales Score  Customer Score  Avg Price Score
Store1 Store2                                              
88.0   1.0       -0.008900        0.038584         0.219070
       2.0        0.023862        0.132533         0.550978
       3.0        0.439119        0.429472         0.196824
       4.0        0.011417       -0.044971         0.217053
       5.0        0.006044        0.030745         0.085577
       6.0        0.157099        0.139840         0.246329
       7.0        0.029080        0.199702         0.114574
       8.0        0.568149        0.551868         0.070527
       9.0        0.102152        0.107250         0.254165
       10.0       0.294613        0.416613         0.083916
       12.0      -0.048957        0.027944        -0.029100
       13.0       0.338147        0.541279         0.034269
       15.0       0.094182        0.293649         0.084397
       16.0       0.076561        0.119802         0.196710
       17.0       0.466696        0.419600         0.422660

In [305]:
score_control['Final'] = 0.5 * (score_control['Sales Score'] + score_control['Customer Score']) #+ score_control['Avg Price Score'])

score_control.head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
88.0   1.0       -0.008900        0.038584         0.219070  0.014842
       2.0        0.023862        0.132533         0.550978  0.078198
       3.0        0.439119        0.429472         0.196824  0.434295
       4.0        0.011417       -0.044971         0.217053 -0.016777
       5.0        0.006044        0.030745         0.085577  0.018394

In [306]:
score_control.sort_values(by='Final', ascending=False).head()

Sales Score  Customer Score  Avg Price Score     Final
Store1 Store2                                                        
88.0   187.0      0.781949        0.600441        -0.013934  0.691195
       57.0       0.743818        0.601630         0.290520  0.672724
       60.0       0.669133        0.673298         0.070588  0.671215
       77.0       0.743402        0.581507         0.107780  0.662454
       23.0       0.633568        0.642813        -0.153108  0.638190

In [308]:
control_sore = 237

past_Sales = metric_data.copy()

store_type = []

for i in past_Sales['STORE_NBR']:
    if i == trial_store:
        store_type.append('Trial Store')
    elif i == control_sore:
        store_type.append('Control Store')
    else:
        store_type.append('Other Stores')
        
past_Sales['Store_Type'] = store_type

past_Sales['MONTH'] = pd.to_datetime(past_Sales['MONTH_YEAR'])

trial_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Total Sales']]
trial_88.set_index('MONTH', inplace=True)
trial_88.rename(columns={'Total Sales':'Trial Store'}, inplace=True)

control_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Total Sales']]
control_88.set_index('MONTH', inplace=True)
control_88.rename(columns={'Total Sales':'Control Store'}, inplace=True)

others_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Total Sales']]
others_88 = pd.DataFrame(others_88.groupby('MONTH')['Total Sales'].mean())
others_88.rename(columns={'Total Sales':'Other Stores'}, inplace=True)

combine_sales = pd.concat([control_88, trial_88, others_88], axis=1)

combine_sales.to_csv('Combines_Sales_88.csv', index=True)

In [309]:
trial_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', ['MONTH', 'Customer_per_store']]
trial_88.set_index('MONTH', inplace=True)
trial_88.rename(columns={'Customer_per_store':'Trial Store'}, inplace=True)

control_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', ['MONTH', 'Customer_per_store']]
control_88.set_index('MONTH', inplace=True)
control_88.rename(columns={'Customer_per_store':'Control Store'}, inplace=True)

others_88 = past_Sales.loc[past_Sales['Store_Type'] == 'Other Stores', ['MONTH', 'Customer_per_store']]
others_88 = pd.DataFrame(others_88.groupby('MONTH')['Customer_per_store'].mean())
others_88.rename(columns={'Customer_per_store':'Other Stores'}, inplace=True)

combine_customer = pd.concat([control_88, trial_88, others_88], axis=1)

combine_customer.to_csv('Combines_Cusotmers_88.csv', index=True)

Assessment for Store 88 

In [310]:
trial_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Total Sales'].sum()
control_sum = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Total Sales'].sum()
scaling_sales = trial_sum/control_sum
scaling_sales

0.9487787449655708

In [311]:
scaled_control_stores = metric.copy()

scaled_control_stores = scaled_control_stores.loc[scaled_control_stores['STORE_NBR'] == control_sore]
scaled_control_stores

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price
2496,237,2018-07,468.2,108,47,9.961702,2.297872,4.335185
2497,237,2018-08,386.5,95,52,7.432692,1.826923,4.068421
2498,237,2018-09,410.8,94,44,9.336364,2.136364,4.370213
2499,237,2018-10,425.7,95,46,9.254348,2.065217,4.481053
2500,237,2018-11,537.2,128,58,9.262069,2.206897,4.196875
2501,237,2018-12,478.2,114,52,9.196154,2.192308,4.194737
2502,237,2019-01,372.2,86,41,9.078049,2.097561,4.327907
2503,237,2019-02,404.0,92,42,9.619048,2.190476,4.391304
2504,237,2019-03,444.6,104,49,9.073469,2.122449,4.275000
2505,237,2019-04,434.6,98,48,9.054167,2.041667,4.434694


In [312]:
scaled_control_stores['Control Sales'] = scaled_control_stores['Total Sales']*scaling_sales


percent_off = scaled_control_stores[['MONTH_YEAR', 'Control Sales']]
percent_off.reset_index(drop=True, inplace=True)

trial_sales = metric.loc[metric['STORE_NBR'] == trial_store, 'Total Sales']
trial_sales.reset_index(drop=True, inplace=True)

percent_off = pd.concat([percent_off, trial_sales], axis=1)
percent_off.rename(columns={'Total Sales':'Trial Sales'}, inplace=True)

percent_off

,MONTH_YEAR,Control Sales,Trial Sales
0,2018-07,444.218208,447.80
1,2018-08,366.702985,417.90
2,2018-09,389.758308,356.80
3,2018-10,403.895112,350.00
4,2018-11,509.683942,435.40
5,2018-12,453.705996,500.80
6,2019-01,353.135449,412.40
7,2019-02,383.306613,393.20
8,2019-03,421.827030,520.40
9,2019-04,412.339243,468.60


In [313]:
percent_off['Percentage_diff'] = abs(percent_off['Control Sales'] - percent_off['Trial Sales'])/percent_off['Control Sales']


std_dev = stdev(percent_off.loc[percent_off['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])


percent_off['tvalue'] = percent_off['Percentage_diff']/std_dev
percent_off.loc[(percent_off['MONTH_YEAR'] > '2019-01') & (percent_off['MONTH_YEAR'] < '2019-05'), 'tvalue']  

7    0.483476
8    4.377233
9    2.555802
Name: tvalue, dtype: float64

In [314]:
t.isf(0.05, dof)

1.943180280392782

In [315]:
scaled_control_stores['MONTH'] = pd.to_datetime(scaled_control_stores['MONTH_YEAR'])
scaled_control_stores

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Sales,MONTH
2496,237,2018-07,468.2,108,47,9.961702,2.297872,4.335185,444.218208,2018-07-01
2497,237,2018-08,386.5,95,52,7.432692,1.826923,4.068421,366.702985,2018-08-01
2498,237,2018-09,410.8,94,44,9.336364,2.136364,4.370213,389.758308,2018-09-01
2499,237,2018-10,425.7,95,46,9.254348,2.065217,4.481053,403.895112,2018-10-01
2500,237,2018-11,537.2,128,58,9.262069,2.206897,4.196875,509.683942,2018-11-01
2501,237,2018-12,478.2,114,52,9.196154,2.192308,4.194737,453.705996,2018-12-01
2502,237,2019-01,372.2,86,41,9.078049,2.097561,4.327907,353.135449,2019-01-01
2503,237,2019-02,404.0,92,42,9.619048,2.190476,4.391304,383.306613,2019-02-01
2504,237,2019-03,444.6,104,49,9.073469,2.122449,4.275000,421.827030,2019-03-01
2505,237,2019-04,434.6,98,48,9.054167,2.041667,4.434694,412.339243,2019-04-01


In [316]:
controlSales = scaled_control_stores.loc[:, ['MONTH', 'Control Sales']]
controlSales.set_index('MONTH', inplace=True)
controlSales

,Control Sales
MONTH,
2018-07-01,444.218208
2018-08-01,366.702985
2018-09-01,389.758308
2018-10-01,403.895112
2018-11-01,509.683942
2018-12-01,453.705996
2019-01-01,353.135449
2019-02-01,383.306613
2019-03-01,421.827030


In [317]:
# Let's calculate the 5% and 95% confidence interval

controlSales['Control 5% Confidence Interval'] = controlSales['Control Sales'] * (1 - std_dev*2)
controlSales['Control 95% Confidence Interval'] = controlSales['Control Sales'] * (1 + std_dev*2)


trial_Sales = percent_off[['MONTH_YEAR', 'Trial Sales']]
trial_Sales['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trial_Sales.set_index('MONTH', inplace=True)

trial_Sales.drop(['MONTH_YEAR'], axis=1, inplace=True)

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\LENOVO\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [318]:
Combined_88 = pd.merge(controlSales, trial_Sales, right_index=True, left_index=True)

In [319]:
Combined_88.to_csv('Test_88.csv', index=True)

In [320]:
trial_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Trial Store', 'Customer_per_store'].sum()
control_customers = past_Sales.loc[past_Sales['Store_Type'] == 'Control Store', 'Customer_per_store'].sum()
scaling_customers = trial_customers/control_customers
scaling_customers

0.9529411764705882

In [321]:
scaled_control_customers = metric.copy()

scaled_control_customers = scaled_control_customers.loc[scaled_control_customers['STORE_NBR'] == control_sore]


scaled_control_customers['Control Customers'] = scaled_control_stores['Customer_per_store']*scaling_customers
scaled_control_customers.head()

,STORE_NBR,MONTH_YEAR,Total Sales,Quantity,Customer_per_store,Transaction_per_customer,Chips_per_customer,Averagr Price,Control Customers
2496,237,2018-07,468.2,108,47,9.961702,2.297872,4.335185,44.788235
2497,237,2018-08,386.5,95,52,7.432692,1.826923,4.068421,49.552941
2498,237,2018-09,410.8,94,44,9.336364,2.136364,4.370213,41.929412
2499,237,2018-10,425.7,95,46,9.254348,2.065217,4.481053,43.835294
2500,237,2018-11,537.2,128,58,9.262069,2.206897,4.196875,55.270588


In [322]:
percent_offc = scaled_control_customers[['MONTH_YEAR', 'Control Customers']]
percent_offc.reset_index(drop=True, inplace=True)

trial_customers = metric.loc[metric['STORE_NBR'] == trial_store, 'Customer_per_store']
trial_customers.reset_index(drop=True, inplace=True)

percent_offc = pd.concat([percent_offc, trial_customers], axis=1)
percent_offc.rename(columns={'Customer_per_store':'Trial Customers'}, inplace=True)

percent_offc

,MONTH_YEAR,Control Customers,Trial Customers
0,2018-07,44.788235,51
1,2018-08,49.552941,45
2,2018-09,41.929412,36
3,2018-10,43.835294,40
4,2018-11,55.270588,49
5,2018-12,49.552941,56
6,2019-01,39.070588,47
7,2019-02,40.023529,46
8,2019-03,46.694118,61
9,2019-04,45.741176,50


In [323]:
percent_offc['Percentage_diff'] = abs(percent_offc['Control Customers'] - 
                                      percent_offc['Trial Customers'])/percent_offc['Control Customers']
percent_offc

,MONTH_YEAR,Control Customers,Trial Customers,Percentage_diff
0,2018-07,44.788235,51,0.138692
1,2018-08,49.552941,45,0.091880
2,2018-09,41.929412,36,0.141414
3,2018-10,43.835294,40,0.087493
4,2018-11,55.270588,49,0.113453
5,2018-12,49.552941,56,0.130104
6,2019-01,39.070588,47,0.202951
7,2019-02,40.023529,46,0.149324
8,2019-03,46.694118,61,0.306374
9,2019-04,45.741176,50,0.093107


In [324]:
std_devc = stdev(percent_offc.loc[percent_offc['MONTH_YEAR'] < '2019-02', 'Percentage_diff'])


percent_offc['tvalue'] = percent_offc['Percentage_diff']/std_devc
percent_offc.loc[(percent_offc['MONTH_YEAR'] > '2019-01') & (percent_offc['MONTH_YEAR'] < '2019-05'), 'tvalue']  

scaled_control_customers['MONTH'] = pd.to_datetime(scaled_control_customers['MONTH_YEAR'])

controlCustomers= scaled_control_customers.loc[:, ['MONTH', 'Control Customers']]
controlCustomers.set_index('MONTH', inplace=True)

In [325]:
controlCustomers

,Control Customers
MONTH,
2018-07-01,44.788235
2018-08-01,49.552941
2018-09-01,41.929412
2018-10-01,43.835294
2018-11-01,55.270588
2018-12-01,49.552941
2019-01-01,39.070588
2019-02-01,40.023529
2019-03-01,46.694118


In [326]:
# Let's calculate the 5% and 95% confidence interval

controlCustomers['Control 5% Confidence Interval'] = controlCustomers['Control Customers'] * (1 - std_devc*2)
controlCustomers['Control 95% Confidence Interval'] = controlCustomers['Control Customers'] * (1 + std_devc*2)
controlCustomers

,Control Customers,Control 5% Confidence Interval,Control 95% Confidence Interval
MONTH,,,
2018-07-01,44.788235,41.307576,48.268894
2018-08-01,49.552941,45.701999,53.403883
2018-09-01,41.929412,38.670922,45.187901
2018-10-01,43.835294,40.428692,47.241897
2018-11-01,55.270588,50.975307,59.565870
2018-12-01,49.552941,45.701999,53.403883
2019-01-01,39.070588,36.034269,42.106908
2019-02-01,40.023529,36.913153,43.133906
2019-03-01,46.694118,43.065345,50.322890


In [328]:
trialCustomers = percent_offc[['MONTH_YEAR', 'Trial Customers']]
trialCustomers['MONTH'] = pd.to_datetime(trial['MONTH_YEAR'])
trialCustomers.set_index('MONTH', inplace=True)
trialCustomers.drop(['MONTH_YEAR'], axis=1, inplace=True)

Combined_88c = pd.merge(controlCustomers, trialCustomers, right_index=True, left_index=True)
Combined_88c

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Control Customers,Control 5% Confidence Interval,Control 95% Confidence Interval,Trial Customers
MONTH,,,,
2018-07-01,44.788235,41.307576,48.268894,51
2018-08-01,49.552941,45.701999,53.403883,45
2018-09-01,41.929412,38.670922,45.187901,36
2018-10-01,43.835294,40.428692,47.241897,40
2018-11-01,55.270588,50.975307,59.565870,49
2018-12-01,49.552941,45.701999,53.403883,56
2019-01-01,39.070588,36.034269,42.106908,47
2019-02-01,40.023529,36.913153,43.133906,46
2019-03-01,46.694118,43.065345,50.322890,61


In [329]:
Combined_88c.to_csv('Cusotmers_88c.csv', index=True)